In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
from numba import jit

from scripts import m5_common

pd.options.display.max_columns = 50

Path and other variables

In [ ]:
path = Path('/kaggle/m5_forecasting/')
assert(path.exists())

In [ ]:
h = 28 
max_lags = h * 2 + 1
tr_last = 1913
fday = datetime(2016, 4, 25)

In [ ]:
cal = pd.read_csv(path/"calendar.csv", dtype = m5_common.CAL_DTYPES)

In [ ]:
%%time

prices, cal = m5_common.prepare_tables(path)

In [ ]:
cal

In [ ]:
prices

In [ ]:
event_name_1_map, event_type_1_map = m5_common.replace_cal_cols(cal)

In [ ]:
uint8_types= ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'month', 'wday', 'weekday', 
              'snap_CA', 'snap_TX', 'snap_WI']
m5_common.convert_uint8(cal, uint8_types)

In [ ]:
m5_common.add_days_before(cal)

In [ ]:
FIRST_DAY = 1 # If you want to load all the data set it to '1' -->  Great  memory overflow  risk !

In [ ]:
%%time

df = m5_common.create_dt(cal, prices, is_train=True, first_day=FIRST_DAY, tr_last=tr_last, path=path)

In [ ]:
df.info()

In [ ]:
df

In [ ]:
def extract_by_state(df):
    df_TX = df[df['state_id'] == m5_common.state_map['TX']]
    df_WI = df[df['state_id'] == m5_common.state_map['WI']]
    df_CA = df[df['state_id'] == m5_common.state_map['CA']]
    return df_TX, df_WI, df_CA

In [ ]:
df_TX.shape, df_WI.shape, df_CA.shape

In [ ]:
assert len(df_TX) + len(df_WI) + len(df_CA) == len(df)

In [ ]:
del df
gc.collect()